In [ ]:
import csv
import pandas as pd
import numpy as np
import pprint

In [ ]:
flows_to_Marseille = []
rank_Marseille = {}
with open('../../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['destination_function'] == "O" \
        and row["source_suite"] == "la Santé registre de patentes de Marseille":
           # and row['toponyme_fr'] == 'Marseille' \
            flows_to_Marseille.append(row)
            rank_Marseille[row["source_doc_id"]] = row["travel_rank"]


In [ ]:
len(flows_to_Marseille)

In [ ]:
ranks_smaller_than_Marseille = []
counter_uhgs_99999 = 0
with open('../../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row["source_suite"] == "la Santé registre de patentes de Marseille":
            if row["destination_uhgs_id"] == 'A9999997':
                counter_uhgs_99999 += 1
                continue
            if row["source_doc_id"] in rank_Marseille:
                max_rank = rank_Marseille[row["source_doc_id"]]
                if row["travel_rank"] <= max_rank:
                    ranks_smaller_than_Marseille.append(row)
counter_uhgs_99999

In [ ]:
from collections import defaultdict
travels = defaultdict(lambda: {"total_miles": 0, "total_steps": 0, "keep": True})
null_distance = 0

for row in ranks_smaller_than_Marseille:
    doc_id = row["source_doc_id"]
    if doc_id == "00182452":
        print(row["travel_rank"])
    travel = travels[doc_id]
    distance = row["distance_dep_dest_miles"]
    if distance and distance != '0' and travel["keep"]:
        
        travel["total_miles"] += int(distance)
        travel["total_steps"] += 1
        if row["travel_rank"] == "1":
            travel["departure_date"] = row["outdate_fixed"]
        if row["travel_rank"] == rank_Marseille[doc_id]:
            travel["arrival_date"] = row["indate_fixed"] 
            travel["pavillon"] = row["ship_flag_standardized_fr"]
            travel["classe_bateau"] = row["ship_class_standardized"]
            year = row["indate_fixed"][:4]
            travel["year"] = year if year[-1] == "9" else year[:3]+"9"
    else:
        travel["keep"] = False



In [ ]:
from datetime import datetime

good_travels = travels.copy()
for k, v in travels.items():
    if not v["keep"]:
        good_travels.pop(k)
    else:
        v["duration"] = (datetime.strptime(v["arrival_date"], "%Y-%m-%d") - datetime.strptime(v["departure_date"], "%Y-%m-%d")).days
        if v["duration"] == 0:
            v["duration"] = 1
        v["speed"] = v["total_miles"] / v["duration"]
        v["decade"] = v["arrival_date"][:4]
        v.pop("keep")


In [ ]:
df_travels = pd.DataFrame(good_travels).transpose()
df_travels = df_travels[df_travels.duration > 0]


In [ ]:
# Vitesse moyenne pour chaque pavillon et chaque décennie
mean_speed = pd.pivot_table(df_travels, values='speed', index=['pavillon'], columns=['year'], aggfunc=np.mean).sort_index()
mean_speed.to_csv("../../data/travel_mean_speed_per_flag_and_year.csv")

In [ ]:
# Nombre de trajets total pour chaque pavillon et chaque décennie
count = pd.pivot_table(df_travels, values='speed', index=['pavillon'], columns=['year'], aggfunc=np.ma.count).sort_index()
count.to_csv("../../data/travel_count_per_flag_and_year.csv")

In [ ]:
# Nombre moyen de flows des trajets pour chaque pavillon et chaque décennie
mean_flow_count = pd.pivot_table(df_travels, values='total_steps', index=['pavillon'], columns=['year'], aggfunc=np.mean).sort_index()
mean_flow_count.to_csv("../../data/mean_flow_count_per_flag_and_year.csv")